In [1]:
import os
import sys
os.chdir('/home/peitian_zhang/Codes/NR')
sys.path.append('/home/peitian_zhang/Codes/NR')

import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import GloVe
from torch.utils.data import DataLoader
from utils.MIND import MIND_iter,MIND_map
from utils.utils import getVocab,getLoss,getLabel,constructBasicDict,run_eval,run_train
from models.FIM import FIMModel

## setting up the *FIM* model

### define paths and hyperparameters, load data

all these hyper parameters are fixed according to the paper [\[29\] Fine-grained Interest Matching for Neural News Recommendation](https://www.aclweb.org/anthology/2020.acl-main.77.pdf)

- *mode*: data to read (*demo*/*small*/*large*)

- *batch_size*: size of each minibatch

- *title_size*: max word capacity of title

- *his_size*: max record capacity of click history

- *npratio*: number of negtive sampling

- *dilation_level*: levels of diferrent dilation rate

- *kernel_size*: size of 1dCNN kernel

- *filter_num*: number of kernels in 1D CNN, which is also embedding dimension of news/user

- *embedding_dim*: word embedding dimension

- *metrics*: metrics to be used in evaluating

- *gpu*: gpu ID if available

- *attrs*: the columns used to construct vocab

### Note 

Although I list *dilation_level* and *kernel_size*, some properties related to both variables are fixed in the FIM model because I don't have time to well-design my model to make it adapt to these variables dynamically. 

In [2]:
hparams = {
    'mode':'demo',
    'npratio':4,
    'batch_size':5,
    'title_size':18,
    'his_size':50,
    'kernel_size':3,
    'dilation_level':3,
    'filter_num':150,
    'embedding_dim':300,
    'metrics':'group_auc,ndcg@4,mean_mrr',
    'gpu':'cuda:0',
    'attrs': ['title','category','subcategory']
}

# customize your path here

news_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/news.tsv'
news_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/news.tsv'
news_file_pair = (news_file_train,news_file_test)

behavior_file_train = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_train/behaviors.tsv'
behavior_file_test = '/home/peitian_zhang/Data/MIND/MIND'+hparams['mode']+'_dev/behaviors.tsv'
behavior_file_pair = (behavior_file_train,behavior_file_test)

save_path = '/home/peitian_zhang/Codes/NR/models/model_param/FIM_'+ hparams['mode'] +'.model'

# if user2id,word2id,news2id haven't been constructed
# if not os.path.exists('data/nid2idx_{}_{}.json'.format(hparams['mode'],'train')):
#     constructBasicDict(news_file_train,behavior_file_train,hparams['mode'],'train',hparams['attrs'])

if not os.path.exists('data/dictionaries/vocab_{}_{}.pkl'.format(hparams['mode'],'_'.join(hparams['attrs']))):
    constructBasicDict(news_file_pair,behavior_file_pair,hparams['mode'],hparams['attrs'])

device = torch.device(hparams['gpu']) if torch.cuda.is_available() else torch.device("cpu")

dataset_train = MIND_map(hparams=hparams,news_file=news_file_train,behaviors_file=behavior_file_train)

dataset_test = MIND_iter(hparams=hparams,news_file=news_file_train,behaviors_file=behavior_file_train)

vocab = dataset_train.vocab
embedding = GloVe(dim=300,cache='.vector_cache')
vocab.load_vectors(embedding)

loader_train = DataLoader(dataset_train,batch_size=hparams['batch_size'],shuffle=True,pin_memory=True,num_workers=3,drop_last=True)
loader_test = DataLoader(dataset_test,batch_size=hparams['batch_size'],pin_memory=True,num_workers=0,drop_last=True)

In [8]:
# you can load my model or train yours

# fimModel = FIMModel(vocab=vocab,hparams=hparams).to(device)
# fimModel.load_state_dict(torch.load(save_path))
# fimModel.train()

fimModel = FIMModel(vocab=vocab,hparams=hparams).to(device)
fimModel.train()

FIMModel(
  (CNN_d1): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(1,))
  (CNN_d2): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
  (CNN_d3): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  (ReLU): ReLU()
  (LayerNorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
  (SeqCNN3D): Sequential(
    (0): Conv3d(3, 32, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (1): ReLU()
    (2): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
    (3): Conv3d(32, 16, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
    (4): ReLU()
    (5): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
  )
  (predictor): Linear(in_features=320, out_features=1, bias=True)
)

### train the model

In [9]:
if fimModel.training:
    print("training...")
    loss_func = getLoss(fimModel)
    optimizer = optim.Adam(fimModel.parameters(),lr=0.001)
    fimModel = run_train(fimModel,loader_train,optimizer,loss_func, epochs=10, interval=15)

training...
epoch 0 , step 390 , loss: 0.1369: : 400it [00:19, 20.76it/s]
epoch 1 , step 120 , loss: 0.1222: : 135it [00:06, 20.52it/s]


KeyboardInterrupt: 

### test & evaluate

In [6]:
fimModel.eval()
fimModel.cdd_size = 1

run_eval(fimModel,loader_test)

15062it [01:36, 156.61it/s]


{'group_auc': 0.5094, 'ndcg@4': 0.2192, 'mean_mrr': 0.2289}

### save the model

In [15]:
fimModel.npratio = 4
torch.save(fimModel.state_dict(), save_path)